## Overview

In [2]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import combinations
from collections import Counter
import sqlite3

## Data Loading
df = pd.read_csv("OnlineRetail.csv", encoding="ISO-8859-1")

print(f"Shape: {df.shape}")
print("\nFirst few rows:")
print(df.head)

Shape: (541909, 8)

First few rows:
<bound method NDFrame.head of        InvoiceNo StockCode                          Description  Quantity  \
0         536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1         536365     71053                  WHITE METAL LANTERN         6   
2         536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3         536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4         536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   
...          ...       ...                                  ...       ...   
541904    581587     22613          PACK OF 20 SPACEBOY NAPKINS        12   
541905    581587     22899         CHILDREN'S APRON DOLLY GIRL          6   
541906    581587     23254        CHILDRENS CUTLERY DOLLY GIRL          4   
541907    581587     23255      CHILDRENS CUTLERY CIRCUS PARADE         4   
541908    581587     22138        BAKING SET 9 PIECE RETROSPOT          3   

         

In [3]:
## Initial Exploration

print("\nColumn Info:")
print(df.info())
 
print("\nBasic Statistics:")
print(df.describe())

print("\nMissing Values:")
print(df.isnull().sum())

#Check unique values in key columns
print(f"\nUnique Invoices: {df['InvoiceNo'].nunique()}")
print(f"Unique Products: {df['StockCode'].nunique()}")
print(f"Unique Customers: {df['CustomerID'].nunique()}")



Column Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB
None

Basic Statistics:
            Quantity      UnitPrice     CustomerID
count  541909.000000  541909.000000  406829.000000
mean        9.552250       4.611114   15287.690570
std       218.081158      96.759853    1713.600303
min    -80995.000000  -11062.060000   12346.000000
25%         1.000000       1.250000   13953.000000
50%         3.000000       2.080000   15152.000000
75

In [4]:
## Data cleaning

print(f"Original dataset size: {len(df)}")

# Remove rows with missing cusomter ID

df_clean = df.dropna(subset=['CustomerID'])
print(f"Size after missing CustomerID: {len(df_clean)}")

# Remove cancelled orders
df_clean = df_clean[~df_clean['InvoiceNo'].astype(str).str.startswith('C')]
print(f"After removing cancelled orders: {len(df_clean)}")

# Remove rows with negative or zero quantities 
df_clean = df_clean[df_clean['Quantity'] > 0]
print (f"After removing negative/zero quantities: len{df_clean}")

# Remove rows with negative prices 
df_clean = df_clean[df_clean['UnitPrice'] > 0]
print (f"After removing negative prices: {len(df_clean)}")

# Remove rows with missing Description
df_clean = df_clean.dropna(subset=['Description'])
print(f"After removing missing descriptions: {len(df_clean)}")

# Stripping whitespace from Description and converting to uppercase
df_clean['Description'] = df_clean['Description'].str.strip().str.upper()

print(f"\nFinal cleaned dataset size: {len(df_clean)}")
print(f"Percentage retained: {len(df_clean)/len(df)*100:.2f}%")

Original dataset size: 541909
Size after missing CustomerID: 406829
After removing cancelled orders: 397924
After removing negative/zero quantities: len       InvoiceNo StockCode                          Description  Quantity  \
0         536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1         536365     71053                  WHITE METAL LANTERN         6   
2         536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3         536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4         536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   
...          ...       ...                                  ...       ...   
541904    581587     22613          PACK OF 20 SPACEBOY NAPKINS        12   
541905    581587     22899         CHILDREN'S APRON DOLLY GIRL          6   
541906    581587     23254        CHILDRENS CUTLERY DOLLY GIRL          4   
541907    581587     23255      CHILDRENS CUTLERY CIRCUS PARADE         4   
5